<a href="https://colab.research.google.com/github/Rishiatweb/Chronos-data-Scribe/blob/main/LAD_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers scikit-learn pandas numpy

In [ ]:
from google.colab import files
import io
print("Please upload your log file(e.g. app.log, server.txt): ")
uploaded = files.upload()
if not uploaded:
  print("\nNo file uploaded. Please upload a file to proceed.")
  log_lines = []
  log_filename = None
else:
  log_filename = list(uploaded.keys())[0]
  print(f"\nUploaded '{log_filename}' ({len(uploaded[log_filename])} bytes)")
  # Read the log lines
  log_content = uploaded[log_filename].decode('utf-8', errors='ignore') # Decode, ignore errors for robustness
  log_lines = log_content.splitlines()
  log_lines = [line.strip() for line in log_lines if line.strip()] # Remove empty lines and leading/trailing whitespace
if log_lines:
  print(f"Read {len(log_lines)} non-empty log lines.")
  # Display a few lines for verification
  print("\nFirst few log lines:")
  for i, line in enumerate(log_lines[:5]):
    print(f"{i+1}: {line}")
elif log_filename: # File was uploaded but resulted in no lines
    print("The uploaded file contains no processable log lines after stripping.")

Please upload your log file(e.g. app.log, server.txt): 


Saving LOG(1).txt to LOG(1) (2).txt

Uploaded 'LOG(1) (2).txt' (10169 bytes)
Read 122 non-empty log lines.

First few log lines:
1: 2023-10-26 10:00:01 INFO: Application startup initiated. Version 1.2.3
2: 2023-10-26 10:00:02 INFO: Configuration loaded from /etc/app/config.xml
3: 2023-10-26 10:00:03 DEBUG: Database connection pool initialized with 10 connections.
4: 2023-10-26 10:00:04 INFO: Listening on port 8080
5: 2023-10-26 10:01:15 INFO: User 'alice' logged in successfully from IP 192.168.1.10


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
if log_lines:
  print("\nLoading sentence transformer model...")
  model_name = 'all-MiniLM-L6-v2'
  try:
    embedder = SentenceTransformer(model_name)
    print(f"Model '{model_name}' loaded successfully.")

    print("Generative embeddings for log lines...")
    log_embeddings = embedder.encode(log_lines, show_progress_bar=True)
    print(f"Generated {log_embeddings.shape[0]} embeddings with dimension {log_embeddings.shape[1]}.")
  except Exception as e:
    print(f"Error during model loading or embeddings: {e}")
    log_embeddings = None
else:
  log_embeddings = None
  print("No log lines to process. Skipping embeddings.")


Loading sentence transformer model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model 'all-MiniLM-L6-v2' loaded successfully.
Generative embeddings for log lines...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Generated 122 embeddings with dimension 384.


In [ ]:
from sklearn.ensemble import IsolationForest
if log_embeddings is not None and log_embeddings.shape[0]>0:
  print("\nTraining Isolation Forest for anomaly detection...")
  contamination_rate = 0.1
  if len(log_lines) < 20:
    contamination_rate = 'auto'
  n_estimators_val = 100
  if log_embeddings.shape[0]<50:
    n_estimators_val = log_embeddings.shape[0]//2 if log_embeddings.shape[0] > 10 else 10
  try:
    iso_forest = IsolationForest(n_estimators=n_estimators_val, contamination=contamination_rate, random_state=42)#try including n_jobs = -1#
    iso_forest.fit(log_embeddings)
    predictions = iso_forest.predict(log_embeddings)
    anomalous_indices = np.where(predictions == -1)[0]
    print(f"Isolation Forest identified {len(anomalous_indices)} potential anomalies.")
  except Exception as e:
    print(f"Error during Isolation Forest training or prediction: {e}")
    anomalous_indices = np.array([]) # Empty array if error
    predictions = None

else:
    anomalous_indices = np.array([])
    predictions = None
    print("\nSkipping anomaly detection as no embeddings were generated.")


Training Isolation Forest for anomaly detection...
Isolation Forest identified 13 potential anomalies.


In [ ]:
if log_lines and predictions is not None:
    print("\n--- Log Analysis Results ---")
    normal_count = 0
    anomaly_count = 0
    for i, line in enumerate(log_lines):
        if predictions[i] == -1:
            print(f"[ANOMALY] {i+1}: {line}")
            anomaly_count +=1
        else:
            # Optionally print normal lines too, or just anomalies
            print(f"[NORMAL]  {i+1}: {line}")
            normal_count +=1
    print(f"\n--- Summary ---")
    print(f"Total lines processed: {len(log_lines)}")
    print(f"Normal lines: {normal_count}")
    print(f"Potential anomalies detected: {anomaly_count}")

elif log_lines and predictions is None and log_embeddings is not None:
    print("\nAnomaly detection could not be performed due to an error.")
elif not log_lines:
    print("\nNo log data to analyze.")


--- Log Analysis Results ---
[NORMAL]  1: 2023-10-26 10:00:01 INFO: Application startup initiated. Version 1.2.3
[NORMAL]  2: 2023-10-26 10:00:02 INFO: Configuration loaded from /etc/app/config.xml
[NORMAL]  3: 2023-10-26 10:00:03 DEBUG: Database connection pool initialized with 10 connections.
[ANOMALY] 4: 2023-10-26 10:00:04 INFO: Listening on port 8080
[NORMAL]  5: 2023-10-26 10:01:15 INFO: User 'alice' logged in successfully from IP 192.168.1.10
[NORMAL]  6: 2023-10-26 10:01:17 DEBUG: User 'alice' session created: session_abc123
[NORMAL]  7: 2023-10-26 10:01:20 INFO: Request received: GET /api/data?id=123 user='alice'
[NORMAL]  8: 2023-10-26 10:01:21 DEBUG: Querying database for item id=123
[NORMAL]  9: 2023-10-26 10:01:22 INFO: Data retrieval successful for item id=123. Rows: 1
[NORMAL]  10: 2023-10-26 10:01:25 INFO: User 'bob' logged in successfully from IP 192.168.1.12
[NORMAL]  11: 2023-10-26 10:01:27 DEBUG: User 'bob' session created: session_def456
[NORMAL]  12: 2023-10-26 1

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine

# This cell assumes the previous cells have been run and the following variables exist:
# - log_lines: The list of all log messages.
# - log_embeddings: The embeddings for all log lines.
# - predictions: The prediction array (-1 for anomaly, 1 for normal) from Isolation Forest.
# - anomalous_indices: The indices of the anomalous logs.

if 'anomalous_indices' in locals() and anomalous_indices.size > 0:
    print("\n" + "="*25)
    print("  Anomaly Explanations")
    print("="*25 + "\n")
    print("An anomaly is a log line that is semantically different from the majority of 'normal' logs.\n"
          "We will now compare each anomaly against the most typical log patterns found in your file.\n")

    # 1. Separate normal and anomalous embeddings
    normal_indices = np.where(predictions == 1)[0]
    normal_embeddings = log_embeddings[normal_indices]

    if normal_embeddings.shape[0] > 0:
        # 2. Find the "center of normality" by averaging the normal embeddings
        center_of_normalcy = np.mean(normal_embeddings, axis=0)

        # 3. Find the most typical normal logs (closest to the center)
        # We use cosine distance: a smaller value means more similar (closer).
        distances_from_center = [cosine(emb, center_of_normalcy) for emb in normal_embeddings]

        # Get the indices of the top 5 closest normal logs
        num_typical_examples = min(5, len(normal_indices))
        most_typical_indices_in_normal_set = np.argsort(distances_from_center)[:num_typical_examples]

        # Map these back to their original indices in `log_lines`
        most_typical_original_indices = normal_indices[most_typical_indices_in_normal_set]

        # 4. For each anomaly, explain why it's an anomaly by showing the contrast
        for i, anomaly_index in enumerate(anomalous_indices):
            anomaly_log = log_lines[anomaly_index]

            print(f"\n--- Analysis for Anomaly #{i+1} ---")
            print(f"  [ANOMALY LOG] Line {anomaly_index + 1}: {anomaly_log}")
            print("\n  [REASONING]")
            print("  This log is considered an anomaly because its content is semantically distant")
            print("  from the common patterns found in the normal logs. It represents a rare or")
            print("  unexpected event compared to the typical behavior.")

            print("\n  [COMPARISON]")
            print(f"  For context, here are {num_typical_examples} of the most 'normal' log lines from your file:")
            for original_idx in most_typical_original_indices:
                print(f"    - (Line {original_idx + 1}) {log_lines[original_idx]}")
    else:
        print("Could not perform analysis because no 'normal' logs were identified to establish a baseline.")

elif 'anomalous_indices' in locals() and anomalous_indices.size == 0:
    print("\n--- No Anomalies to Explain ---")
    print("The model did not flag any log lines as anomalies based on the current settings.")

else:
    print("\nAnalysis could not be performed. Please ensure the anomaly detection cell ran successfully.")


  Anomaly Explanations

An anomaly is a log line that is semantically different from the majority of 'normal' logs.
We will now compare each anomaly against the most typical log patterns found in your file.


--- Analysis for Anomaly #1 ---
  [ANOMALY LOG] Line 4: 2023-10-26 10:00:04 INFO: Listening on port 8080

  [REASONING]
  This log is considered an anomaly because its content is semantically distant
  from the common patterns found in the normal logs. It represents a rare or
  unexpected event compared to the typical behavior.

  [COMPARISON]
  For context, here are 5 of the most 'normal' log lines from your file:
    - (Line 110) 2023-10-26 10:39:05 INFO: Cache flush completed.
    - (Line 115) 2023-10-26 10:40:04 INFO: Final log entry for this batch, signing off.
    - (Line 17) 2023-10-26 10:03:05 INFO: Request received: GET /api/data?id=456 user='bob'
    - (Line 38) 2023-10-26 10:08:00 INFO: Request received: GET /api/data?id=000 user='bob'
    - (Line 105) 2023-10-26 10:37

In [ ]:
# --- Step 1: Install the necessary Hugging Face libraries ---
!pip install -q transformers accelerate

from transformers import pipeline
import torch
from IPython.display import display, Markdown
import textwrap

# --- Step 2: Set up the Hugging Face Pipeline ---
print("Setting up Hugging Face text-generation pipeline...")

if torch.cuda.is_available():
    device = 0
    print("GPU is available. The model will run on the GPU.")
else:
    device = -1
    print("GPU not available. The model will run on the CPU (this will be significantly slower).")

model_name = "google/flan-t5-base"
try:
    # We only load the generator once
    if 'generator' not in locals():
        print(f"Loading model '{model_name}'...")
        generator = pipeline('text2text-generation', model=model_name, device=device)
        print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    generator = None

# --- Step 3: Analyze Anomalies Using the Local Model with a Better Prompt ---
if generator and 'anomalous_indices' in locals() and anomalous_indices.size > 0:
    print("\n" + "="*50)
    print("  LLM-Powered Anomaly Explanations (Hugging Face)")
    print("="*50 + "\n")

    normal_indices = np.where(predictions == 1)[0]
    num_normal_samples = min(10, len(normal_indices))

    if num_normal_samples > 0:
        sample_normal_indices = np.random.choice(normal_indices, num_normal_samples, replace=False)
        normal_log_samples = "\n".join([log_lines[i] for i in sample_normal_indices])

        # --- THIS IS THE NEW, "FEW-SHOT" PROMPT ---
        # We provide a perfect example to teach the model how to behave.
        few_shot_prompt_template = """
### INSTRUCTIONS ###
You are an expert log analysis assistant. Your task is to explain why a given log line is an anomaly.
First, look at the examples of normal logs to understand the baseline behavior. Then, analyze the anomalous log and explain the difference.
Provide a concise, structured analysis.

### EXAMPLE ###
[NORMAL logs]
2023-11-15 10:00:05 INFO: User 'alice' logged in successfully.
2023-11-15 10:01:12 INFO: User 'bob' accessed the dashboard.
2023-11-15 10:02:00 INFO: User 'charlie' updated their profile.

[ANOMALOUS log]
2023-11-15 10:03:45 CRITICAL: Database connection failed: timeout expired.

[ANALYSIS]
The normal logs show routine user activity like logins and page access. The anomalous log is a CRITICAL system-level failure indicating a complete loss of database connectivity. This is an anomaly because it represents a service outage, not standard user behavior.

### TASK ###
[NORMAL logs]
{normal_logs_text}

[ANOMALOUS log]
{anomaly_log_text}

[ANALYSIS]
"""

        for i, anomaly_index in enumerate(anomalous_indices):
            anomaly_log = log_lines[anomaly_index]

            print(f"\n--- Analyzing Anomaly #{i+1} / {len(anomalous_indices)} ---")
            display(Markdown(f"**Anomalous Log (Line {anomaly_index + 1}):** `{anomaly_log}`"))

            # Populate the template with the actual log data
            final_prompt = few_shot_prompt_template.format(
                normal_logs_text=normal_log_samples,
                anomaly_log_text=anomaly_log
            )

            # Generate the explanation.
            # We use `max_new_tokens` to give it enough space to write a good answer.
            # We remove `max_length` to avoid the warning message.
            outputs = generator(final_prompt, max_new_tokens=256)
            explanation = outputs[0]['generated_text']

            display(Markdown(textwrap.fill(explanation, width=100)))

    else:
        print("Could not generate explanations as no 'normal' logs were found to create a baseline.")

elif not generator:
    print("Could not proceed with analysis because the language model failed to load.")
else:
    print("\nNo anomalies to analyze or previous cells were not run.")

Setting up Hugging Face text-generation pipeline...
GPU is available. The model will run on the GPU.

  LLM-Powered Anomaly Explanations (Hugging Face)


--- Analyzing Anomaly #1 / 13 ---


**Anomalous Log (Line 4):** `2023-10-26 10:00:04 INFO: Listening on port 8080`

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


An anomalous log is a system-level failure indicating a complete loss of database connectivity. This
is an anomaly because it represents a service outage, not standard user behavior.


--- Analyzing Anomaly #2 / 13 ---


...

**Anomalous Log (Line 95):** `2023-10-26 10:31:00 WARN: Certificate for 'external.partner.api.com' expiring in 7 days.`

The anomalous log is a CRITICAL system-level failure indicating a complete loss of database
connectivity. This is an anomaly because it represents a service outage, not standard user behavior.


--- Analyzing Anomaly #13 / 13 ---


**Anomalous Log (Line 98):** `2023-10-26 10:32:10 INFO: Search for 'rare_item_name' yielded 0 results.`

Anomalous logs show routine user activity like logins and page access. This is an anomaly because it
represents a complete loss of database connectivity. This is an anomaly because it represents a
service outage, not standard user behavior.